In [1]:
import re
import json
import os
from typing import Optional, List, Any, Dict, Set, Tuple
import numpy as np
from copy import deepcopy
import random

In [2]:
def read_json(fname: str) -> Any:
    """
    Given a filename, reads a json file and returns the data stored inside.

    Input:
        fname (str):
            Name of the file to be read.

    Output:
        data (Any):
            The data loaded from the json file.
    """

    assert os.path.isfile(fname)
    assert fname.endswith(".json")

    with open(fname, "r") as file:
        data = json.load(file)

    return data


def write_json(
    data: Any,
    fname: str,
) -> None:
    """
    Given a data and the filename, writes the data to the specified
    fname.
    If the directory that the specified filename should be in
    does not exist, then it creates the directory first.

    Input:
        data (Any):
            the data that needs to stored in a json format.

        fname (str):
            path to the file where the data needs to be saved.

    Output:
        None
    """

    assert isinstance(fname, str) and fname.endswith(".json")
    splits = fname.split("/")[:-1]
    root_dir = "/".join(splits)
    if not os.path.isdir(root_dir):
        os.makedirs(root_dir, exist_ok=True)

    with open(fname, "w") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [3]:
def check_all_letters_unique(
    word: str,
) -> bool:
    word = word.upper().strip()
    assert word.isalpha()
    
    return len(set([letter for letter in word])) == len(word)

In [4]:
def generate_jotto_dataset(
    wordle_config_path: str,
    wordle_modified_config_path: str,
    jotto_config_path: str,
    num_train_examples: int,
) -> None:
    wordle_data = read_json(fname=wordle_config_path)
    wordle_modified_data = read_json(fname=wordle_modified_config_path)
    
    print(wordle_data.keys())
    print(wordle_modified_data.keys())
    
    train_data = wordle_data["train"] + wordle_modified_data["eval"]
    test_data = wordle_data["eval"]
    
    train_data_filtered = []
    all_words_already_seen = set()
    for data in train_data:
        curr_word = data["env"]
        if (
            check_all_letters_unique(word=curr_word)
            and curr_word not in all_words_already_seen
        ):
            train_data_filtered.append(data)
            all_words_already_seen.add(curr_word)
            
    test_data_filtered = []
    for data in test_data:
        curr_word = data["env"]
        if (
            check_all_letters_unique(word=data["env"])
            and curr_word not in all_words_already_seen
        ):
            test_data_filtered.append(data)
            all_words_already_seen.add(curr_word)
            
    print(len(train_data_filtered))
    print(len(test_data_filtered))
            
            
    total_data = train_data_filtered + test_data_filtered
    assert len(total_data) > num_train_examples
    
    train_indices = np.random.choice(
        a=len(total_data),
        size=num_train_examples,
        replace=False,
    ).tolist()
    
    test_indices = [
        i for i in range(len(total_data)) if i not in train_indices
    ]
    
    train_data_final = [
        total_data[index]
        for index in train_indices
    ]
    
    test_data_final = [
        total_data[index]
        for index in test_indices
    ]
    
    print("final train data: ", len(train_data_final))
    print("final test data: ", len(test_data_final))
            
            
    final_data = {
        "train": train_data_final,
        "test": test_data_final,
    }
    
    write_json(
        data=final_data,
        fname=jotto_config_path,
    )

In [5]:
generate_jotto_dataset(
    wordle_config_path="/Users/fahimtajwar/academics/llm_exploration/llm_exploration/game/game_configs/wordle.json",
    wordle_modified_config_path="/Users/fahimtajwar/academics/llm_exploration/llm_exploration/game/game_configs/wordle_modified.json",
    jotto_config_path="/Users/fahimtajwar/academics/exploration/verl/verl/paprika/environments/env_configs/jotto.json",
    num_train_examples=1252,
)

dict_keys(['env', 'agent', 'environment_default_response', 'judge_prompt_agent', 'judge_prompt_env', 'env_optional_message', 'judge_prompt_suffix', 'agent_optional_message', 'max_turns', 'train', 'eval'])
dict_keys(['env', 'agent', 'environment_default_response', 'judge_prompt_agent', 'judge_prompt_env', 'env_optional_message', 'judge_prompt_suffix', 'agent_optional_message', 'max_turns', 'eval'])
1206
546
final train data:  1252
final test data:  500
